In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csics2017/cleanDatasetWithoutEncoding.csv


In [2]:
dataset = pd.read_csv('../input/csics2017/cleanDatasetWithoutEncoding.csv')

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
dataset.head()

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
0,49188.0,4.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal
1,49486.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal
2,88.0,609.0,7.0,4.0,484.0,414.0,233.0,0.0,69.142860,111.967896,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal
3,88.0,879.0,9.0,4.0,656.0,3064.0,313.0,0.0,72.888885,136.153810,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal
4,88.0,1160.0,9.0,6.0,3134.0,3048.0,1552.0,0.0,348.222230,682.482540,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal


## Train Test Split

In [5]:
features = dataset.loc[:, dataset.columns != 'Label'].astype('float64')
labels = dataset['Label']
features.head()

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,act_data_pkt_fwd,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin
0,49188.0,4.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49486.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,88.0,609.0,7.0,4.0,484.0,414.0,233.0,0.0,69.142860,111.967896,...,5.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,88.0,879.0,9.0,4.0,656.0,3064.0,313.0,0.0,72.888885,136.153810,...,7.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,88.0,1160.0,9.0,6.0,3134.0,3048.0,1552.0,0.0,348.222230,682.482540,...,7.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=.3, random_state=42)

## Standardization

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

## LASSO Feature Selection

In [8]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [9]:
lsvc = LinearSVC(C=0.1, penalty="l1",max_iter=4000, dual=False).fit(features_train_scaled, labels_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [10]:
sel_ = SelectFromModel(lsvc, prefit=True)

In [11]:
sel_.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [12]:
print('features_train.columns')
print(features_train.columns)
selected_feat = features_train.columns[(sel_.get_support())]
print('total features: {}'.format((features_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))

features_train.columns
Index(['DestinationPort', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytess', 'FlowPacketss', 'FlowIATMean', 'FlowIATStd', 'FlowIATMax',
       'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd', 'FwdIATMax',
       'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd', 'BwdIATMax',
       'BwdIATMin', 'FwdPSHFlags', 'FwdURGFlags', 'FwdHeaderLength',
       'BwdHeaderLength', 'FwdPacketss', 'BwdPacketss', 'MinPacketLength',
       'MaxPacketLength', 'PacketLengthMean', 'PacketLengthStd',
       'PacketLengthVariance', 'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount',
       'PSHFlagCount', 'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount',
       'ECEFlagCount', 'DownUpRatio', 'Ave

In [13]:
removed_feats = list(set(features_train.columns)-set(selected_feat))
print("Folowing are the features which got removed by Lasso L1 regularization")
print(removed_feats)

Folowing are the features which got removed by Lasso L1 regularization
['BwdHeaderLength', 'FwdHeaderLength', 'FwdHeaderLength.1']
